In [1]:
import gym
import click
import multiagent
import time
import tensorflow as tf
import make_env
import numpy as np
from rl.common.misc_util import set_global_seeds
from sandbox.mack.acktr_disc import Model, onehot
from sandbox.mack.acktr_disc_om import Model as Model_om
from sandbox.mack.policies import CategoricalPolicy
from sandbox.mack.policies_om import CategoricalPolicy as CategoricalPolicy_om
from sandbox.mack.opponent_policies import CategoricalPolicy as oppo_CategoricalPolicy
from rl import bench
import imageio
import seaborn as sns
import pickle as pkl
from sklearn.neighbors.kde import KernelDensity
import argparse

C:\Users\Renos\anaconda3\envs\TFCW\lib\site-packages\sklearn\utils\deprecation.py:144: FutureWarning: The sklearn.neighbors.kde module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [2]:
#@click.command()
#@click.option('--image', is_flag=True, flag_value=True)
#@click.option('--vis_dis', is_flag=True, flag_value=True)
#@click.option('--hyper_study', is_flag=True, flag_value=True)
#@click.option('--d', type=click.INT, default=1)
#@click.option('--g', type=click.INT, default=1)
#@click.option('--ent_coef', type=click.FLOAT, default=0)
#@click.option('--kl', is_flag=True, flag_value=True)
#@click.option('--env', type=click.STRING, default='simple_speaker_listener')
#@click.option('--algo', type=click.STRING, default='mack')
#@click.option('--disc_type', type=click.STRING, default='decentralized')
# @click.option('--path', type=click.STRING, default='./results/mack/simple_speaker_listener/l-0.1-b-1000/seed-1/checkpoint07000')
#@click.option('--seed', type=click.INT, default=1)
#@click.option('--epoch', type=click.INT, default=55000)
#@click.option('--sample', is_flag=True, flag_value=True)
#@click.option('--all_exp', is_flag=True, flag_value=True)
#@click.option('--curve', is_flag=True, flag_value=True)
#@click.option('--num_trajs', type=click.INT, default=100)
#@click.option('--traj_limitation', type=click.INT, default=200)


arg_image = True
arg_env = 'simple_tag'
arg_algo = 'mack'
arg_seed = 1
arg_epoch = 55000
arg_path = "tbd"
arg_num_trajs = 100

In [3]:
env_name = arg_env
env_id = arg_env

def create_env(seed):
    env = make_env.make_env(env_id)
    env.seed(seed)
    # env = bench.Monitor(env, '/tmp/',  allow_early_resets=True)
    set_global_seeds(seed)
    return env

env = create_env(0)
n_agents = len(env.action_space)

ob_space = env.observation_space
ac_space = env.action_space

n_actions = [action.n for action in ac_space]

WARN: Could not seed environment <MultiAgentEnv instance>



In [4]:
path = './results/'+arg_algo+'/'+env_name+'/'
if arg_algo in ['mack', 'mppo', 'mack_om']:
    path += '/l-0.1-b-1000/seed-'+str(1)+'/checkpoint' + '%05d'%arg_epoch
    
print(path)



./results/mack/simple_tag//l-0.1-b-1000/seed-1/checkpoint55000


In [5]:
make_model = lambda: Model(
    CategoricalPolicy, ob_space, ac_space, 1, total_timesteps=1e7, nprocs=2, nsteps=500,
    nstack=1, ent_coef=0.01, vf_coef=0.5, vf_fisher_coef=1.0, lr=0.01, max_grad_norm=0.5, kfac_clip=0.001,
    lrschedule='linear', identical=make_env.get_identical(env_id), use_kfac=False)
if arg_algo in ['mack_om', 'codail']:
    print("here for some reason")
    make_model = lambda: Model_om(
        CategoricalPolicy_om, oppo_CategoricalPolicy, ob_space, ac_space, 1, total_timesteps=1e7, nprocs=2, nsteps=500,
        nstack=1, ent_coef=0.01, vf_coef=0.5, vf_fisher_coef=1.0, lr=0.01, max_grad_norm=0.5, kfac_clip=0.001,
        lrschedule='linear', identical=make_env.get_identical(env_id), use_kfac=False)

model = make_model()
model.load(path)

images = []
sample_trajs = []
# num_trajs = 100
max_steps = 50
avg_ret = [[] for _ in range(n_agents)]

episode_r_n_all = []
episode_r_n_sum = []

all_pos = [[] for k in range(n_agents)]



[1, 2, 3, 4]



Tensor("policy_0/pi/add:0", shape=(1, 5), dtype=float32) Tensor("Placeholder_16:0", shape=(1,), dtype=int32)


Tensor("policy_0_1/pi/add:0", shape=(500, 5), dtype=float32) Tensor("Placeholder_20:0", shape=(500,), dtype=int32)
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead

Tensor("policy_1/pi/add:0", shape=(1, 5), dtype=float32) Tensor("Placeholder_24:0", shape=(1,), dtype=int32)
Tensor("policy_1_1/pi/add:0", shape=(500, 5), dtype=float32) Tensor("Placeholder_28:0", shape=(500,), dtype=int32)
Tensor("policy_2/pi/add:0", shape=(1, 5), dtype=float32) Tensor("Placeholder_32:0", shape=(1,), dtype=int32)
Tensor("policy_2_1/pi/add:0", shape=(500, 5), dtype=float32) Tensor("Placeholder_36:0", shape=(500,), dtype=int32)
Tensor("policy_3/pi/add:0", shape=(1, 5), dtype=float32) Tensor("Placeholder_40:0", shape=(1,), dtype=int32)
Tensor("policy_3_1/pi/add:0", shape=(500, 5), dtype=

In [6]:
obs = env.reset()

In [7]:
for i in range(arg_num_trajs):
    print("trajectory ", i)
    all_ob, all_agent_ob, all_ac, all_rew, ep_ret = [], [], [], [], [0 for k in range(n_agents)]
    for k in range(n_agents):
        all_ob.append([])
        all_ac.append([])
        all_rew.append([])
    obs = env.reset()
    obs = [ob[None, :] for ob in obs]
    action = [np.zeros([1]) for _ in range(n_agents)]
    step = 0
    done = False
    episode_r_n = [0.] * n_agents
    while (not done) or (step<=2):
        if arg_algo != "random":
            action, _, _ = model.step(obs, action)
            actions_list = [onehot(action[k][0], n_actions[k]) for k in range(n_agents)]
        else:
            actions_list = [onehot(np.random.randint(n_actions[k]), n_actions[k]) for k in range(n_agents)]

        for k in range(n_agents):
            all_ob[k].append(obs[k])
            all_ac[k].append(actions_list[k])
        all_agent_ob.append(np.concatenate(obs, axis=1))
        obs, rew, done, _ = env.step(actions_list)
        print(obs)
        episode_r_n = [episode_r_n[i]+rew[i] for i in range(n_agents)]
        for k in range(n_agents):
            all_rew[k].append(rew[k])
            ep_ret[k] += rew[k]
            all_pos[k].append(np.array(env.world.agents[k].state.p_pos))

        obs = [ob[None, :] for ob in obs]
        step += 1

        if arg_image:
            img = env.render(mode='rgb_array')
            images.append(img[0])
            time.sleep(0.02)
        if step == max_steps or True in done:
            done = True
            # step = 0
        else:
            done = False

    # print("\n--- episode-{} | [sum-reward]: {}".format(i, np.sum(episode_r_n)))
    episode_r_n_all.append(episode_r_n)
    episode_r_n_sum.append(np.sum(episode_r_n))

    for k in range(n_agents):
        all_ob[k] = np.squeeze(all_ob[k])

    all_agent_ob = np.squeeze(all_agent_ob)
    traj_data = {
        "ob": all_ob, "ac": all_ac, "rew": all_rew,
        "ep_ret": ep_ret, "all_ob": all_agent_ob
    }

    sample_trajs.append(traj_data)
    # print('episode', i, 'expected_return', ep_ret)

    for k in range(n_agents):
        avg_ret[k].append(ep_ret[k])

trajectory  0
[array([-2.15736223e-01, -1.49373438e-02, -2.59484589e-01, -6.63638830e-02,
        2.44884729e-01,  7.96152668e-03,  6.02217684e-01,  2.89494674e-01,
       -6.68517214e-01,  8.94693437e-01,  5.08651230e-01,  6.02327227e-01,
        4.65795022e-01,  8.69924469e-01,  4.00000000e-01,  1.28813471e-82]), array([ 3.00000000e-01,  0.00000000e+00, -9.28001803e-01,  8.28329554e-01,
        9.13401942e-01, -8.86731910e-01,  1.27073490e+00, -6.05198763e-01,
        6.68517214e-01, -8.94693437e-01,  1.17716844e+00, -2.92366210e-01,
        1.13431224e+00, -2.47689676e-02,  4.00000000e-01,  1.28813471e-82]), array([-3.53506267e-13,  3.00000000e-01,  2.49166641e-01,  5.35963344e-01,
       -2.63766502e-01, -5.94365700e-01,  9.35664534e-02, -3.12832553e-01,
       -5.08651230e-01, -6.02327227e-01, -1.17716844e+00,  2.92366210e-01,
       -4.28562086e-02,  2.67597242e-01,  4.00000000e-01,  1.28813471e-82]), array([ 4.00000000e-01,  1.28813471e-82,  2.06310432e-01,  8.03560586e-01,
    

[0. 0.]
[array([ 0.04182276, -0.52261289, -0.28448335, -0.21264251,  0.26988349,
        0.15424015,  0.62721644,  0.4357733 ,  0.03561183,  1.08711464,
        1.02482354,  0.78395517,  1.16391802,  0.58229685,  0.28958588,
       -1.2203125 ]), array([ 0.83623238, -0.15380859, -0.24887152,  0.87447213,  0.23427166,
       -0.93287449,  0.59160461, -0.65134134, -0.03561183, -1.08711464,
        0.98921171, -0.30315948,  1.12830619, -0.5048178 ,  0.28958588,
       -1.2203125 ]), array([ 0.36275482, -0.61783104,  0.74034019,  0.57131266, -0.75494006,
       -0.62971501, -0.3976071 , -0.34818186, -1.02482354, -0.78395517,
       -0.98921171,  0.30315948,  0.13909448, -0.20165832,  0.28958588,
       -1.2203125 ]), array([ 0.28958588, -1.2203125 ,  0.87943467,  0.36965434, -0.89403453,
       -0.42805669, -0.53670158, -0.14652355, -1.16391802, -0.58229685,
       -1.12830619,  0.5048178 , -0.13909448,  0.20165832])]
adsdfasdf
[0. 0.]
[array([ 0.03136707, -0.69195967, -0.28134664, -0.2818

[array([ 0.39432206,  0.60195442,  0.32976686, -0.5500127 , -0.34436672,
        0.49161035,  0.01296623,  0.77314349, -0.34752912,  0.78736294,
        0.27761976,  0.15855239,  0.12199461,  0.1379017 , -0.45424066,
        0.75161976]), array([ 0.0658682 , -0.43006896, -0.01776226,  0.23735024,  0.0031624 ,
       -0.2957526 ,  0.36049536, -0.01421945,  0.34752912, -0.78736294,
        0.62514888, -0.62881055,  0.46952373, -0.64946124, -0.45424066,
        0.75161976]), array([-0.81104213, -0.31627041,  0.60738662, -0.39146031, -0.62198648,
        0.33305796, -0.26465352,  0.6145911 , -0.27761976, -0.15855239,
       -0.62514888,  0.62881055, -0.15562515, -0.02065069, -0.45424066,
        0.75161976]), array([-0.45424066,  0.75161976,  0.45176147, -0.412111  , -0.46636133,
        0.35370864, -0.10902837,  0.63524179, -0.12199461, -0.1379017 ,
       -0.46952373,  0.64946124,  0.15562515,  0.02065069])]
adsdfasdf
[0. 0.]
[array([ 0.29574154,  0.75146581,  0.35934102, -0.47486612, -0

[array([ 0.43158057,  0.48971892,  0.73767728,  0.21832598, -0.75227714,
       -0.27672834, -0.39494418,  0.00480481, -0.82412131,  0.61015598,
        0.07040175,  0.19381565,  0.00131015,  0.54623908, -0.52853631,
        0.71086339]), array([ 0.23124036,  0.74684142, -0.08644403,  0.82848197,  0.07184417,
       -0.88688432,  0.42917713, -0.60535118,  0.82412131, -0.61015598,
        0.89452306, -0.41634034,  0.82543146, -0.0639169 , -0.52853631,
        0.71086339]), array([ 0.26190682,  0.96509317,  0.80807902,  0.41214163, -0.82267888,
       -0.47054399, -0.46534593, -0.18901084, -0.07040175, -0.19381565,
       -0.89452306,  0.41634034, -0.0690916 ,  0.35242343, -0.52853631,
        0.71086339]), array([-0.52853631,  0.71086339,  0.73898742,  0.76456506, -0.75358728,
       -0.82296742, -0.39625433, -0.54143427, -0.00131015, -0.54623908,
       -0.82543146,  0.0639169 ,  0.0690916 , -0.35242343])]
adsdfasdf
[0. 0.]
[array([ 0.32368543,  0.66728919,  0.77004582,  0.2850549 , -0

[array([-9.68538986e-01, -2.48861876e-01, -1.33785079e-01,  4.12964250e-01,
        1.19185218e-01, -4.71366607e-01,  4.76518173e-01, -1.89833459e-01,
        2.89223672e-02,  2.71252477e-01,  4.10758714e-01,  2.17940367e-01,
       -3.05786441e-01,  3.67034432e-04, -7.27893542e-01, -7.69982521e-01]), array([-0.98137776,  0.18254836, -0.10486271,  0.68421673,  0.09026285,
       -0.74261908,  0.44759581, -0.46108594, -0.02892237, -0.27125248,
        0.38183635, -0.05331211, -0.33470881, -0.27088544, -0.72789354,
       -0.76998252]), array([-0.10372195, -0.28775153,  0.27697363,  0.63090462, -0.2915735 ,
       -0.68930697,  0.06575946, -0.40777383, -0.41075871, -0.21794037,
       -0.38183635,  0.05331211, -0.71654516, -0.21757333, -0.72789354,
       -0.76998252]), array([-7.27893542e-01, -7.69982521e-01, -4.39571520e-01,  4.13331285e-01,
        4.24971660e-01, -4.71733641e-01,  7.82304615e-01, -1.90200494e-01,
        3.05786441e-01, -3.67034432e-04,  3.34708809e-01,  2.70885443e-

...

[array([-0.0219427 , -0.48149554,  1.00521456,  0.53377653, -0.27577483,
       -0.30516217, -1.17280967, -0.47621356,  0.10982782, -0.26330075,
       -1.06451799, -1.07453195, -0.20963041, -1.07834178,  0.3149231 ,
        0.34740753]), array([ 0.51320781, -0.55056282,  1.11504238,  0.27047578, -0.38560266,
       -0.04186143, -1.28263749, -0.21291281, -0.10982782,  0.26330075,
       -1.17434581, -0.81123121, -0.31945824, -0.81504104,  0.3149231 ,
        0.34740753]), array([ 0.41533656, -0.69197296, -0.05930343, -0.54075542,  0.78874315,
        0.76936978, -0.10829168,  0.5983184 ,  1.06451799,  1.07453195,
        1.17434581,  0.81123121,  0.85488757, -0.00380983,  0.3149231 ,
        0.34740753]), array([ 0.3149231 ,  0.34740753,  0.79558414, -0.54456525, -0.06614442,
        0.77317961, -0.96317926,  0.60212823,  0.20963041,  1.07834178,
        0.31945824,  0.81504104, -0.85488757,  0.00380983])]
adsdfasdf
[0. 0.]
[array([ 0.28354298, -0.36112166,  1.03356886,  0.49766436, -0

[array([-0.9996358 ,  0.0269866 ,  0.15237876,  0.56805883,  0.57706097,
       -0.33944447, -0.31997387, -0.51049585,  0.31613531, -0.44225614,
       -0.04703249, -0.29909151, -0.19737643, -0.12723765, -1.12330465,
        0.65380073]), array([-0.20458394,  0.79498911,  0.46851406,  0.12580268,  0.26092566,
        0.10281167, -0.63610918, -0.06823971, -0.31613531,  0.44225614,
       -0.3631678 ,  0.14316463, -0.51351173,  0.31501849, -1.12330465,
        0.65380073]), array([-0.4750986 ,  0.29491905,  0.10534627,  0.26896731,  0.62409346,
       -0.04035295, -0.27294138, -0.21140434,  0.04703249,  0.29909151,
        0.3631678 , -0.14316463, -0.15034394,  0.17185386, -1.12330465,
        0.65380073]), array([-1.12330465,  0.65380073, -0.04499767,  0.44082117,  0.77443739,
       -0.21220682, -0.12259744, -0.3832582 ,  0.19737643,  0.12723765,
        0.51351173, -0.31501849,  0.15034394, -0.17185386])]
adsdfasdf
[0. 0.]
[array([-0.99981417,  0.01927758,  0.05239734,  0.56998658,  0

[array([-0.17763957, -0.98409562, -0.79415544,  0.15091276,  1.52359517,
        0.0777016 ,  0.62656033, -0.09334978,  1.20421286,  0.10707987,
        0.02525551, -0.18802231, -0.2296644 , -0.36562733, -0.11516812,
       -1.29488853]), array([-0.00455879, -0.95486188,  0.41005741,  0.25799262,  0.31938231,
       -0.02937827, -0.57765253, -0.20042965, -1.20421286, -0.10707987,
       -1.17895735, -0.29510217, -1.43387726, -0.4727072 , -0.11516812,
       -1.29488853]), array([-0.177537  , -0.97389714, -0.76889994, -0.03710955,  1.49833966,
        0.26572391,  0.60130482,  0.09467252, -0.02525551,  0.18802231,
        1.17895735,  0.29510217, -0.25491991, -0.17760503, -0.11516812,
       -1.29488853]), array([-0.11516812, -1.29488853, -1.02381985, -0.21471458,  1.75325957,
        0.44332893,  0.85622473,  0.27227755,  0.2296644 ,  0.36562733,
        1.43387726,  0.4727072 ,  0.25491991,  0.17760503])]
adsdfasdf
[0. 0.]
[array([-0.12729927, -0.99186435, -0.80688537,  0.05172632,  1

trajectory  31
[array([ 3.00000000e-01,  0.00000000e+00, -9.64746072e-01, -8.32546519e-01,
        1.07409954e+00,  5.48492152e-01,  8.25891287e-01,  1.22305012e+00,
        1.62246798e+00,  1.63301363e+00,  5.48167620e-01,  1.28305810e+00,
        9.56457166e-01,  1.45369438e+00,  5.12110696e-25, -4.00000000e-01]), array([-3.00000000e-001,  2.04100230e-257,  6.57721909e-001,
        8.00467114e-001, -5.48368444e-001, -1.08452148e+000,
       -7.96576695e-001, -4.09963516e-001, -1.62246798e+000,
       -1.63301363e+000, -1.07430036e+000, -3.49955529e-001,
       -6.66010815e-001, -1.79319250e-001,  5.12110696e-025,
       -4.00000000e-001]), array([ 3.00000000e-01,  3.57194030e-20, -4.16578452e-01,  4.50511585e-01,
        5.25931917e-01, -7.34565952e-01,  2.77723667e-01, -6.00079876e-02,
       -5.48167620e-01, -1.28305810e+00,  1.07430036e+00,  3.49955529e-01,
        4.08289546e-01,  1.70636278e-01,  5.12110696e-25, -4.00000000e-01]), array([ 5.12110696e-25, -4.00000000e-01, -8.2889

KeyboardInterrupt: 

In [ ]:
obs = env.reset()
img = env.render(mode='rgb_array')

In [ ]:
obs = env.reset()
images = []
for i in range(50):
    
    
    
    img = env.render(mode='rgb_array')
    images.append(img[0])
    time.sleep(0.02)



In [12]:
data_path = './results/'+arg_algo+'/'+env_name+'/l-0.1-b-1000/seed-'+str(1)+'/checkpoint' + '%05d'%arg_epoch + "-%d"%arg_num_trajs + "tra-1.pkl"